To start, each part is rated in each of four categories:

    x: Extremely cool looking
    m: Musical (it makes a noise when you hit it)
    a: Aerodynamic
    s: Shiny

Then, each part is sent through a series of workflows that will ultimately accept or reject the part. Each workflow has a name and contains a list of rules; each rule specifies a condition and where to send the part if the condition is true. The first rule that matches the part being considered is applied immediately, and the part moves on to the destination described by the rule. (The last rule in each workflow has no condition and always applies if reached.)

If a part is sent to another workflow, it immediately switches to the start of that workflow instead and never returns. If a part is accepted (sent to A) or rejected (sent to R), the part immediately stops any further processing.

The workflows are listed first, followed by a blank line, then the ratings of the parts the Elves would like you to sort. All parts begin in the workflow named in

In [6]:
wf_dict = {}
parts = []

cmp = {'>': int.__gt__,
       '<': int.__lt__}

def accept(part, label='in'):
    if label == 'A':
        return True
    if label == 'R':
        return False
    
    rules, default = wf_dict[label]

    for cat, op, val, tgt in rules:
        if cmp[op](part[cat], val):
            return accept(part, tgt)
    return accept(part, default)

blk1, blk2  = open('input.txt').read().split('\n\n')
for line in blk1.splitlines():
    wf, inst = line[:-1].split('{')
    inst = inst.split(',')
    default = inst.pop()
    
    wf_dict[wf] = ([], default)
    for s in inst:
        s = s.split(':')
        cat = s[0][:1]
        op = s[0][1]
        val = int(s[0][2:])
        tgt = s[1]
        wf_dict[wf][0].append((cat, op, val, tgt))

total = 0
for line in blk2.splitlines():
    p = {}
    for s in line[1:-1].split(','):
        cat, val = s.split('=')
        p[cat] = int(val)
    if accept(p):
        total += sum(p.values())

print(total)

319295


Each of the four ratings (x, m, a, s) can have an integer value ranging from a minimum of 1 to a maximum of 4000. Of all possible distinct combinations of ratings, your job is to figure out which ones will be accepted.

In the above example, there are 167409079868000 distinct combinations of ratings that will be accepted.

Consider only your list of workflows; the list of part ratings that the Elves wanted you to sort is no longer relevant. How many distinct combinations of ratings will be accepted by the Elves' workflows?

In [5]:
wf_dict = {}

blk1, blk2  = open('input.txt').read().split('\n\n')
for line in blk1.splitlines():
    wf, inst = line[:-1].split('{')
    inst = inst.split(',')
    default = inst.pop()
    
    wf_dict[wf] = ([], default)
    for s in inst:
        cond, tgt = s.split(':')
        cat = cond[:1]
        op = cond[1]
        val = int(cond[2:])
        wf_dict[wf][0].append((cat, op, val, tgt))

def count(ranges, label = "in"):
    if label == "R":
        return 0
    if label == "A":
        product = 1
        for lo, hi in ranges.values():
            product *= hi - lo + 1
        return product
    rules, default = wf_dict[label]
    total = 0

    for cat, op, val, tgt in rules:
        lo, hi = ranges[cat]
        if op == "<":
            T = (lo, min(val - 1, hi))
            F = (max(val, lo), hi)
        else:
            T = (max(val + 1, lo), hi)
            F = (lo, min(val, hi))
        if T[0] <= T[1]:
            r_copy = dict(ranges)
            r_copy[cat] = T
            total += count(r_copy, tgt)
        if F[0] <= F[1]:
            ranges = dict(ranges)
            ranges[cat] = F
        else:
            break
    else:
        total += count(ranges, default)
            
    return total

print(count({cat: (1, 4000) for cat in "xmas"}))

110807725108076
